In [69]:
import pandas as pd
import numpy as np
from collections import Counter

In [70]:
df = pd.read_csv("sales_2017-01-01_2020-05-07.csv")

In [71]:
df.shape

(13758, 37)

In [4]:
df[df["billing_country"]=="China"].financial_status.value_counts()

paid        10
refunded     4
Name: financial_status, dtype: int64

In [21]:
L = []
for i in df["variant_sku"].unique()[1:]:
    if i is not None:
        L.append(i[0:2])
set(L)

{'10', '20', '21', '22', '24', '40', '41', '42', '90', '94'}

## Group sku
### some of order doesn't have the sku record

In [47]:
df = pd.read_csv("sales_2017-01-01_2020-05-07.csv")

In [79]:
df.loc[df.variant_sku=="False","variant_sku"]= 0 #delete the "FALSE"

In [80]:
df["variant_sku"].value_counts()

21300    845
21200    727
21400    563
41300    460
41200    428
21100    367
21500    339
40600    289
41400    278
20600    255
40500    254
21600    247
40700    224
20700    206
41100    195
20500    194
41500    182
21700    181
41600    142
40400    118
41700    102
20400     93
21800     90
22800     77
22600     58
40300     49
20800     47
24800     46
41800     44
22700     44
40800     37
20300     35
22500     33
24600     28
24700     21
22400     19
24500     19
40200     15
20200     10
22300      4
24400      4
94700      3
24300      3
22200      2
94600      2
42800      1
10500      1
90600      1
90700      1
10700      1
0          1
Name: variant_sku, dtype: int64

In [81]:
df["grouped_product"] = df["variant_sku"].apply(lambda x: int(x[0:2]) if type(x)==str else "")

In [82]:
df["grouped_product"].value_counts()  # makes sense, women are buying a lot.

      6374
21    3359
41    1831
40     986
20     840
22     237
24     121
94       5
90       2
10       2
42       1
Name: grouped_product, dtype: int64

## Create a column of product name

In [83]:
def create_name(group):
    if group == 21:
        name = "Women's +20"
    elif group == 41:
        name = "Women's +40"
    elif group == 40:
        name = "Men's +40"
    elif group == 20:
        name = "Men's +20"
    elif group == 22:
        name = "Petite Women's +20"
    elif group == 24:
        name = "Petite Women's +40"
    else:
        name = ""
    return name

df["product_name"] = df["grouped_product"].apply(lambda x: "Women's +20" if x==21 elif x==41 "Women's +40" 
                                                 elif x==40 "Men's +40" elif x==20 "Men's +20" 
                                                elif x==22 "Petite Women's +20" elif x==24 "Petite Women's +40" else "")
# Why this is wrong???????

In [84]:
df["product_name"] = df["grouped_product"].apply(create_name)

In [85]:
df["product_name"].value_counts()

                      6384
Women's +20           3359
Women's +40           1831
Men's +40              986
Men's +20              840
Petite Women's +20     237
Petite Women's +40     121
Name: product_name, dtype: int64

L = " " * df.shape[0]
df["grouped_product"]= L

df["grouped_product"] #created a column of zeros

for index,row in df.iterrows():
    if type(row["variant_sku"]) == str:
        row["grouped_product"]=int(row["variant_sku"][0:2])

for index,row in df.head().iterrows():
    if type(row["variant_sku"])== str:
        row["grouped_product"] = int(row["variant_sku"][0:2])
    else:
        row["grouped_product"] = 10

df["grouped_product"].value_counts()

Type = []
for index,row in df.iterrows():
    Type.append(type(row["variant_sku"]))
Counter(Type)

for index,row in df.iterrows():
    if type(row["variant_sku"])== NoneType:
        print(row["variant_sku"])

In [86]:
df["customer_id"].value_counts()

0                488
2443837374575     46
708008673391      32
1305334677615     24
708019060847      20
                ... 
2719343607919      1
2706240700527      1
2312866103407      1
2675389694063      1
2447336407151      1
Name: customer_id, Length: 4934, dtype: int64

## Shipping city different from billing city

In [100]:
df_subset = df[df.billing_city != df.shipping_city][["billing_city","shipping_city"]] # different

In [105]:
df_subset.shape

(1814, 2)

In [106]:
sum(pd.isna(df_subset["billing_city"])==False) #682 billing cities exist

682

In [108]:
sum(pd.isna(df_subset["shipping_city"])==False) #1289 shipping cities exist

1289

##### 1814 different, in which only 682 billing city make sense, not null.
##### Some of them just didn't leave billing city message
##### in all different, only 662 make sense, have both billing city and shipping city information

In [87]:
df_diff_ship_bill = df[(df.billing_city != df.shipping_city) & (pd.isna(df.billing_city)==False) &(pd.isna(df.shipping_city)==False)]

In [88]:
df_diff_ship_bill.shape

(662, 39)

In [123]:
# 20 person only leave billing city, no shipping city. interesting
df_empty_ship_with_bill = df[(pd.isna(df.billing_city)==False) & (pd.isna(df.shipping_city)==True)]
df_empty_ship_with_bill[["billing_city","shipping_city"]]

,billing_city,shipping_city
253,Saint Louis,NaN
258,Delray Beach,NaN
261,Richmond,NaN
268,Lakewood,NaN
274,University City,NaN
275,Saint Louis,NaN
276,Saint Louis,NaN
277,Fort Lauderdale,NaN
278,Clayton,NaN
279,Webster Groves,NaN


In [94]:
# Pairs of different billing_city and shipping_city
L = []
i = []
for index,row in df_diff_ship_bill.iterrows():
    i = (row["billing_city"],row["shipping_city"])
    L.append(i)
new_L = map(tuple,L)
Counter(new_L).most_common()

[(('Farmington Hills', 'Harper Woods'), 10),
 (('Brooklyn', 'New York'), 10),
 (('Baltimore', 'Severn'), 9),
 (('St. Louis', 'Kirkwood'), 8),
 (('El Segundo', 'Irvine'), 7),
 (('Dallas', 'Cary'), 6),
 (('Irvine', 'Carlsbad'), 6),
 (('Miami', 'North Miami'), 6),
 (('Rockport', 'Shavano Park'), 5),
 (('Sioux Falls', 'Anaheim'), 5),
 (('Scarsdale', 'Portland'), 5),
 (('Cypress', 'La Mirada'), 5),
 (('Gurnee', 'Milwaukee'), 5),
 (('Oxnard', 'Ventura'), 5),
 (('London', 'Port Huron'), 5),
 (('Spring Hill', 'Land O Lakes'), 5),
 (('Douglasville', 'Rockmart'), 5),
 (('Corvallis', 'Bend'), 4),
 (('Mechanicsburg', 'College Park'), 4),
 (('Union City', 'Norman'), 4),
 (('Florence', 'Hartsville'), 4),
 (('Aurora', 'Estero'), 4),
 (('Spicewood', 'Lake Oswego'), 4),
 (('Swansea', 'Saint Louis'), 4),
 (('Naples', 'Arden'), 4),
 (('Cumming', 'Alpharetta'), 4),
 (('Fairfield', 'Prairie View'), 4),
 (('Boxford', 'Boston'), 4),
 (('La Crescenta Montrose', 'La Crescenta'), 4),
 (('Scottsdale', 'Phoenix, 

In [90]:
len(L)

662

In [95]:
df_diff_ship_bill["pair"]= L

/Users/xinyifang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
df_diff_ship_bill["pair"].value_counts() #correct

(Farmington Hills, Harper Woods)    10
(Brooklyn, New York)                10
(Baltimore, Severn)                  9
(St. Louis, Kirkwood)                8
(El Segundo, Irvine)                 7
                                    ..
(Memphis, Philadelphia)              2
(Lancaster, Palmdale)                2
(Lone Butte, Kent)                   2
(Las Vegas, Saint Petersburg)        2
(Saint Louis, Los Angeles)           1
Name: pair, Length: 256, dtype: int64

In [97]:
df_diff_ship_bill.to_csv("diff_ship_bill_city.csv",index=False)

## Orders - Whether there's any transaction
When units per transaction > 0, orders is 1 <br>
When there's no transaction, order = 0, financial status: 4867 1304 426 3 3 <br>
When there is transaction, order = 1, 5906 544 214 2 2 <br>
status: paid/ refunded/ partially_refunded/ partially_paid/ pending

In [98]:
df["orders"].value_counts()

0    7090
1    6668
Name: orders, dtype: int64

In [134]:
df["units_per_transaction"].value_counts()

1.0     6425
0.0     6373
2.0      204
3.0       19
4.0        8
5.0        7
6.0        3
12.0       2
Name: units_per_transaction, dtype: int64

In [135]:
sum(df["units_per_transaction"]>0)

6668

In [139]:
df[(df["units_per_transaction"]>0) & (df["orders"]==1)].shape

(6668, 39)

In [100]:
df[df["orders"]==0].order_name  #7090 orders=0

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
13749    #7455
13752    #7450
13753    #7451
13754    #7457
13757    #7456
Name: order_name, Length: 7090, dtype: object

In [102]:
df["financial_status"].value_counts()

paid                  10773
refunded               1848
partially_refunded      640
partially_paid            5
pending                   5
Name: financial_status, dtype: int64

In [151]:
#order = 0, financial status
print(df[(df["financial_status"]=="paid") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="refunded") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="pending") & (df["orders"]==0)].shape[0])

4867 1304 426 3 3


In [152]:
#order = 1, financial status
print(df[(df["financial_status"]=="paid") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="refunded") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="pending") & (df["orders"]==1)].shape[0])

5906 544 214 2 2


In [104]:
df["cancelled"].value_counts()

No     13171
Yes      100
Name: cancelled, dtype: int64

In [105]:
df["sale_kind"].value_counts()

order     11762
return     1509
Name: sale_kind, dtype: int64

In [114]:
sum(df.net_sales!=0) #6048 orders has positive net sales, 7063 not zero.

7063

In [111]:
sum(df.total_sales>0)

8368

## Returns - modify by financial status

In [115]:
df["returns"].value_counts()

 0.00      12437
-129.99      415
-109.99      135
 129.99      124
-114.99      110
           ...  
-140.87        1
-104.99        1
-5.67          1
-234.55        1
 219.98        1
Name: returns, Length: 103, dtype: int64

In [116]:
df[df["returns"]>0]["returns"].value_counts()

129.99    124
109.99     61
114.99     36
99.99      27
104.00     16
15.00       7
20.00       6
142.57      6
30.00       3
109.66      3
117.00      2
65.00       2
126.12      2
99.00       1
35.00       1
104.99      1
100.00      1
89.00       1
259.98      1
222.29      1
120.81      1
239.98      1
114.06      1
219.98      1
Name: returns, dtype: int64

In [132]:
# The order is paid but have record of return.
df_return_paid = df[(df.returns!=0) & (df.financial_status == "paid")][["financial_status","returns","net_sales","gross_sales","total_sales","discounts"]]
df_return_paid

,financial_status,returns,net_sales,gross_sales,total_sales,discounts
4357,paid,-114.99,-114.99,0.0,-114.99,0.0
4366,paid,114.99,114.99,0.0,114.99,0.0
4391,paid,-114.99,-114.99,0.0,-114.99,0.0
4394,paid,114.99,114.99,0.0,114.99,0.0
4472,paid,109.99,109.99,0.0,109.99,0.0
...,...,...,...,...,...,...
13543,paid,129.99,129.99,0.0,129.99,0.0
13672,paid,239.98,239.98,0.0,239.98,0.0
13674,paid,99.99,99.99,0.0,99.99,0.0
13696,paid,-239.98,-239.98,0.0,-239.98,0.0


In [126]:
sum(df_return_paid["net_sales"]==df_return_paid.["total_sales"])

218

In [133]:
df_return_paid.gross_sales.value_counts()

0.0    441
Name: gross_sales, dtype: int64

### there are 218 orders with positive refund value and "paid" financial status. So, modify the refund value to be zero. 
### all of them has net sales equal to total_sales

In [130]:
# The order is refunded and have record of return.
df_return_refunded = df[(df.returns<0) & (df.financial_status == "refunded")][["financial_status","returns","net_sales","gross_sales","total_sales","discounts"]]
df_return_refunded.head()

,financial_status,returns,net_sales,gross_sales,total_sales,discounts
2521,refunded,-65.00,-65.00,0.0,-71.30,0.0
2703,refunded,-120.90,-120.90,0.0,-120.90,0.0
2861,refunded,-110.90,-110.90,0.0,-110.90,0.0
2867,refunded,-129.99,-129.99,0.0,-129.99,0.0
3027,refunded,-114.99,-114.99,0.0,-114.99,0.0


## Adjustment
some adjusted order is refunded, some is not. not very meaningful information

In [140]:
df.adjustment.value_counts()

No     12642
Yes      629
Name: adjustment, dtype: int64

In [141]:
df.financial_status.value_counts()

paid                  10773
refunded               1848
partially_refunded      640
partially_paid            5
pending                   5
Name: financial_status, dtype: int64

## financial status and sales kind
Sales kind:<br>
in 11762 order, 10330 records' financial status is paid, 1032 is refunded, 393 partially refunded, 3 partially_paid, 4 pending <br>
in 1509 returns, 443 is paid, 816 is refunded, 247 is partially refunded, 2 is partially paid, 1 is pending

In [149]:
print(df[(df["financial_status"]=="paid") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="refunded") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="pending") & (df["sale_kind"]=="order")].shape[0])

10330 1032 393 3 4


In [150]:
print(df[(df["financial_status"]=="paid") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="refunded") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="pending") & (df["sale_kind"]=="return")].shape[0])

443 816 247 2 1


## Order Quantity
net_quantity:<br>
Record the refunded order as different order from the one that's created to buy and later refunded. <br>
For each refunded order, there's two records: one is buying, one is refunding<br>

ordered_item_quantity:<br>
Only record those with trasaction, refunded is recorded as 0. -> 6668 transactions

In [153]:
df["net_quantity"].value_counts() 

 1     6425
 0     6373
-1      707
 2      204
 3       19
-2        9
 4        8
 5        7
 6        3
 12       2
-12       1
Name: net_quantity, dtype: int64

In [157]:
sum(df["net_quantity"]<0) #there's 717 refund order record.

717

In [154]:
df["ordered_item_quantity"].value_counts() #expect from 0, other numbers are right, consistent with above

0     7090
1     6425
2      204
3       19
4        8
5        7
6        3
12       2
Name: ordered_item_quantity, dtype: int64

In [155]:
sum(df["ordered_item_quantity"]>0) # there's transaction. orders = 1 and units_per_transaction > 0

6668

In [156]:
#ordered_item_quantity = 0, financial status
print(df[(df["financial_status"]=="paid") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="refunded") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="pending") & (df["ordered_item_quantity"]==0)].shape[0])
# same as order = 0. There's no transaction.

4867 1304 426 3 3


In [158]:
sum(df["financial_status"] == "refunded") #924 order refunded and there's 1848 record.

1848

In [159]:
df.to_csv("modified_sales.csv",index=False)